# legends - Extended Feature Selection
* StelllarAlgo Data Science
* Ryan Kazmerik & Nakisa Rad
* Feb 22, 2022

## Hypothesis
Two very important components of a machine learning model are feature selection and feature engineering. Our idea is that adding some more features to the StellarAlgo retention model could improve performance of the model. This notebook will test the standard StellarAlgo retention model features.

## Experiment
This section details our experiment including querying data, data transformations, feature selection and modelling.

In [1]:
import getpass
import pyodbc
import pandas as pd
import warnings
import matplotlib.pyplot as plt

from pycaret.classification import *

warnings.filterwarnings('ignore')

### Let's connect to MSSQL and run a stored proc to get our dataset:

In [2]:
# connect to SQL Server.
SERVER = '34.206.73.189' 
DATABASE = 'datascience' 
USERNAME = 'nrad' 
PASSWORD = getpass.getpass(prompt='Enter your password')
CNXN = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USERNAME+';PWD='+ PASSWORD)

In [3]:
lkupclientid = 26 # legends

cursor = CNXN.cursor()

storedProc = (
    f"""Exec [stlrMILB].[ds].[getRetentionScoringModelData] {lkupclientid}"""
)

df = pd.read_sql(storedProc, CNXN)

# apply some data transformations
df["year"] = pd.to_numeric(df["year"])

CNXN.commit()
cursor.close()

df.head()
#df.info()

,lkupClientId,dimCustomerMasterId,customerNumber,year,productGrouping,totalSpent,recentDate,attendancePercent,renewedBeforeDays,isBuyer,source_tenure,tenure,distToVenue,totalGames,recency,missed_games_1,missed_games_2,missed_games_over_2,click_link,fill_out_form,open_email,send_email,unsubscribe_email,openToSendRatio,clickToSendRatio,clickToOpenRatio,posting_records,resale_records,resale_atp,forward_records,cancel_records,email,inbound_email,inbound_phonecall,inperson_contact,internal_note,left_message,outbound_email,outbound_phonecall,phonecall,text,unknown,gender,childrenPresentInHH,maritalStatus,lengthOfResidenceInYrs,annualHHIncome,education,urbanicity,credits_after_refund,is_Lockdown,NumberofGamesPerSeason,CNTPostponedGames,isNextYear_Buyer
0,26,320260385,170102,2018,Full Season,2128.0,2018-07-02,0.076087,29,TRUE,199,199,3.61,7,25,0,0,2,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,72,None,1
1,26,320260437,203722,2018,Full Season,1018.4,2018-08-25,0.155303,58,TRUE,205,205,7.63,11,4,0,0,6,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,72,None,0
2,26,320260565,143252,2018,Full Season,532.0,2018-08-28,0.171429,91,TRUE,237,237,3.61,7,1,1,0,3,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,72,None,1
3,26,320261287,161990,2018,Full Season,1330.0,2018-08-25,0.202899,120,TRUE,267,267,6.36,15,4,3,1,7,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,72,None,1
4,26,320261301,143717,2018,Full Season,1018.4,2018-08-26,0.223485,127,TRUE,274,274,2.63,18,3,2,1,6,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,72,None,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 54 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   lkupClientId            265 non-null    int64  
 1   dimCustomerMasterId     265 non-null    int64  
 2   customerNumber          265 non-null    object 
 3   year                    265 non-null    int64  
 4   productGrouping         265 non-null    object 
 5   totalSpent              265 non-null    float64
 6   recentDate              265 non-null    object 
 7   attendancePercent       265 non-null    float64
 8   renewedBeforeDays       265 non-null    int64  
 9   isBuyer                 265 non-null    object 
 10  source_tenure           265 non-null    int64  
 11  tenure                  265 non-null    int64  
 12  distToVenue             265 non-null    float64
 13  totalGames              265 non-null    int64  
 14  recency                 265 non-null    in

### Let's also drop the features that only have a single value, as they won't add much differentiation to our model:

In [5]:
for col in df.columns:
    if len(df[col].unique()) == 1:
        df.drop(col,inplace=True,axis=1)
        
df.shape

(265, 19)

### We should also drop features that have a low correlation with the target label as they won't be useful for prediction, we'll only keep features that have a correlation above a set threshold:

In [6]:
cor = df.corr()

threshold = 0.05

#Correlation with output variable
cor_target = abs(cor["isNextYear_Buyer"])

#Selecting highly correlated features
relevant_features = cor_target[cor_target > threshold]

feats = []
for name, val in relevant_features.items():
    feats.append(name)

df_correlated = df[feats]

df_correlated.shape

df_correlated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   dimCustomerMasterId     265 non-null    int64  
 1   year                    265 non-null    int64  
 2   totalSpent              265 non-null    float64
 3   renewedBeforeDays       265 non-null    int64  
 4   source_tenure           265 non-null    int64  
 5   tenure                  265 non-null    int64  
 6   distToVenue             265 non-null    float64
 7   totalGames              265 non-null    int64  
 8   recency                 265 non-null    int64  
 9   missed_games_1          265 non-null    int64  
 10  missed_games_2          265 non-null    int64  
 11  missed_games_over_2     265 non-null    int64  
 12  is_Lockdown             265 non-null    int64  
 13  NumberofGamesPerSeason  265 non-null    int64  
 14  isNextYear_Buyer        265 non-null    in

### Now that we have the right features we can look at the correlations between them, if features are highly correlated with each other it might negatively impact the model:

In [7]:
corr = df_correlated.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

,dimCustomerMasterId,year,totalSpent,renewedBeforeDays,source_tenure,tenure,distToVenue,totalGames,recency,missed_games_1,missed_games_2,missed_games_over_2,is_Lockdown,NumberofGamesPerSeason,isNextYear_Buyer
dimCustomerMasterId,1.00,0.16,0.17,0.28,0.16,0.16,-0.12,0.26,-0.12,0.20,0.20,0.19,0.16,0.09,0.39
year,0.16,1.00,-0.05,-0.01,0.69,0.69,0.03,0.14,0.17,0.12,-0.06,-0.10,0.77,0.75,-0.28
totalSpent,0.17,-0.05,1.00,0.17,-0.02,-0.02,-0.01,0.32,-0.16,-0.03,0.05,0.19,0.01,-0.09,0.23
renewedBeforeDays,0.28,-0.01,0.17,1.00,0.30,0.30,-0.03,0.15,0.13,0.06,0.06,0.01,-0.16,0.14,0.29
source_tenure,0.16,0.69,-0.02,0.30,1.00,1.00,-0.10,0.19,0.20,0.12,-0.07,-0.09,0.41,0.65,-0.14
tenure,0.16,0.69,-0.02,0.30,1.00,1.00,-0.10,0.19,0.20,0.12,-0.07,-0.09,0.41,0.65,-0.14
distToVenue,-0.12,0.03,-0.01,-0.03,-0.10,-0.10,1.00,-0.02,0.03,-0.02,0.03,-0.04,0.06,-0.02,-0.11
totalGames,0.26,0.14,0.32,0.15,0.19,0.19,-0.02,1.00,-0.38,0.60,0.58,0.58,0.04,0.18,0.06
recency,-0.12,0.17,-0.16,0.13,0.20,0.20,0.03,-0.38,1.00,-0.25,-0.30,-0.40,0.01,0.25,-0.08
missed_games_1,0.20,0.12,-0.03,0.06,0.12,0.12,-0.02,0.60,-0.25,1.00,0.53,0.34,0.13,0.04,0.05


### In order to compare two sets of features, we need to create some datasets for training and evalution:


In [8]:
# select % of the data for training
df_train = df_correlated.sample(frac=0.8, random_state=786).reset_index(drop=True)

# create the eval datasets for A and B
df_eval = df_correlated.drop(df_train.index).reset_index(drop=True)

# print out the number of records for training and eval
print('Data for Modeling: ' + str(df_train.shape))
print('Unseen Data For Predictions: ' + str(df_eval.shape), end="\n\n")

Data for Modeling: (212, 15)
Unseen Data For Predictions: (53, 15)



### Now we can model the data using a binary classification prediction for the isnextyear_buyer field to see how likely a customer is to re-purchase.

In [9]:
setup(
    data= df_train, 
    target="isNextYear_Buyer", 
    train_size = 0.80,
    data_split_shuffle=True,
    silent=True,
    numeric_features=[
        "totalSpent",
        "renewedBeforeDays",
        "source_tenure",
        "tenure",
        "distToVenue",
        "totalGames",
        "recency",
        "missed_games_1",
        "missed_games_2",
        "missed_games_over_2",
        "is_Lockdown",
        "NumberofGamesPerSeason"
    ]
)

,Description,Value
0,session_id,2153
1,Target,isNextYear_Buyer
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(212, 15)"
5,Missing Values,False
6,Numeric Features,13
7,Categorical Features,1
8,Ordinal Features,False
9,High Cardinality Features,False


(0      1
 1      0
 2      1
 3      1
 4      0
       ..
 207    0
 208    1
 209    1
 210    1
 211    0
 Name: isNextYear_Buyer, Length: 212, dtype: int64,
 {'USI',
  'X',
  'X_test',
  'X_train',
  '_all_metrics',
  '_all_models',
  '_all_models_internal',
  '_available_plots',
  '_gpu_n_jobs_param',
  '_internal_pipeline',
  '_ml_usecase',
  'create_model_container',
  'data_before_preprocess',
  'display_container',
  'exp_name_log',
  'experiment__',
  'fix_imbalance_method_param',
  'fix_imbalance_param',
  'fold_generator',
  'fold_groups_param',
  'fold_groups_param_full',
  'fold_param',
  'fold_shuffle_param',
  'gpu_param',
  'html_param',
  'imputation_classifier',
  'imputation_regressor',
  'iterative_imputation_iters_param',
  'log_plots_param',
  'logging_param',
  'master_model_container',
  'n_jobs_param',
  'prep_pipe',
  'pycaret_globals',
  'seed',
  'stratify_param',
  'target_param',
  'transform_target_method_param',
  'transform_target_param',
  'y',
  'y_

In [10]:
model_matrix = compare_models(
    fold=10,
    include=["lr", "xgboost"]
)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.7217,0.7886,0.8044,0.7400,0.7663,0.4224,0.4333,0.1770
lr,Logistic Regression,0.5739,0.7275,1.0000,0.5739,0.7290,0.0000,0.0000,0.3550


In [11]:
best_model = create_model(model_matrix)
final_model = finalize_model(best_model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7059,0.6429,0.7000,0.7778,0.7368,0.4056,0.4085
1,0.7059,0.8000,0.9000,0.6923,0.7826,0.3511,0.3812
2,0.7059,0.7429,0.8000,0.7273,0.7619,0.3796,0.3825
3,0.6471,0.7286,0.6000,0.7500,0.6667,0.3014,0.3099
4,0.8235,0.9143,0.8000,0.8889,0.8421,0.6434,0.6480
5,0.6471,0.8000,0.8000,0.6667,0.7273,0.2388,0.2469
6,0.8824,0.9000,1.0000,0.8333,0.9091,0.7463,0.7715
7,0.6471,0.6250,0.7778,0.6364,0.7000,0.2817,0.2901
8,0.7647,0.8750,0.8889,0.7273,0.8000,0.5211,0.5367
9,0.6875,0.8571,0.7778,0.7000,0.7368,0.3548,0.3578


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

## Observations
Here you can document some ideas on the results from above

## Conclusions
Here you can talk about next steps, did the experiment work? If yes, what to do next? If no, why?